## Installing dependencies

In [1]:
!pip install selenium

  Using cached websocket_client-1.8.0-py3-none-any.whl.metadata (8.0 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl.metadata (5.6 kB)
  Using cached pycparser-2.22-py3-none-any.whl.metadata (943 bytes)
   ---------------------------------------- 0.0/9.5 MB ? eta -:--:--
   ----- ---------------------------------- 1.3/9.5 MB 9.6 MB/s eta 0:00:01
   -------------- ------------------------- 3.4/9.5 MB 9.6 MB/s eta 0:00:01
   -------------------- ------------------- 5.0/9.5 MB 8.9 MB/s eta 0:00:01
   ---------------------------- ----------- 6.8/9.5 MB 8.9 MB/s eta 0:00:01
   ------------------------------------ --- 8.7/9.5 MB 8.8 MB/s eta 0:00:01
   ---------------------------------------- 9.5/9.5 MB 8.4 MB/s eta 0:00:00
Using cached websocket_client-1.8.0-py3-none-any.whl (58 kB)
Using cached sniffio-1.3.1-py3-none-any.whl (10 kB)
Using cached wsproto-1.2.0-py3-non

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
httpx 0.27.2 requires anyio, which is not installed.
langsmith 0.1.136 requires requests<3,>=2, which is not installed.
langsmith 0.1.136 requires requests-toolbelt<2.0.0,>=1.0.0, which is not installed.


### importing selenium

In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.chrome.service import Service

###Other imports here

In [6]:
!pip install webdriver_manager
!pip install wget

  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.3 requires tqdm, which is not installed.
huggingface-hub 0.26.1 requires filelock, which is not installed.
huggingface-hub 0.26.1 requires fsspec>=2023.5.0, which is not installed.
huggingface-hub 0.26.1 requires pyyaml>=5.1, which is not installed.
huggingface-hub 0.26.1 requires tqdm>=4.42.1, which is not installed.
langchain 0.3.4 requires aiohttp<4.0.0,>=3.8.3, which is not installed.
langchain 0.3.4 requires numpy<2.0.0,>=1.26.0; python_version >= "3.12", which is not installed.
langchain 0.3.4 requires PyYAML>=5.3, which is not installed.
langchain 0.3.4 requires SQLAlchemy<3,>=1.4, which is not installed.
langchain 0.3.4 requires tenacity!=8.4.0,<10,>=8.1.0, which is not installed.
langsmith 0.1.136 requires requests-toolbelt<2.0.0,>=1.0.0, which is not installed.
nipype 1.9.2 requir

In [11]:
import os
import wget
import time
import json
from urllib.parse import urljoin
from webdriver_manager.chrome import ChromeDriverManager

chromeDriver Path


In [11]:
chrome_driver_path = r"C:\Users\Chetan\chromedriver-win32\chromedriver.exe"

In [12]:
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

missions = ["insat-3d", "insat-3dr","megha-tropiques", "scatsat-1", "kalpana-1", "saral-altika", "oceansat-2", "oceansat-3"]
mission_subpages = ["introduction", "objectives", "spacecraft", "payloads", "references"]

# Base URL
base_url = "https://www.mosdac.gov.in"

# Dictionary to store extracted data
mission_data_store = {}

# for _ in range(1):
for mission in missions:
    mission_url = f"{base_url}/{mission}"
    # mission_url = f"{base_url}/{mission}"
    driver.get(mission_url)
    # driver.get("https://www.mosdac.gov.in/"+missions[6])
    
    try:
        # Wait for the content area to load
        content_area = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, ".field-item.even"))
        )
        print(f"Element found for: {mission}")

        # Extract data from the main mission page
        mission_data = {
            "text": content_area.text,
            "images": [],
            "tables": [],
            "pdfs": []
        }

        # Extract all images within the section
        images = content_area.find_elements(By.TAG_NAME, "img")
        for img in images:
            img_src = urljoin(base_url, img.get_attribute("src"))
            mission_data["images"].append(img_src)

        # Extract all tables
        tables = content_area.find_elements(By.TAG_NAME, "table")
        for table in tables:
            mission_data["tables"].append(table.get_attribute("outerHTML"))  # table parsing will be done later

        # Extract all PDF links
        pdf_links = content_area.find_elements(By.CSS_SELECTOR, "p a[href$='.pdf']")
        for pdf in pdf_links:
            relative_url = pdf.get_attribute("href")
            full_url = urljoin(base_url, relative_url)
            mission_data["pdfs"].append(full_url)
         
         # Store extracted data
        mission_data_store[mission] = mission_data
    
    except Exception as e:
        print(f"Error: Element not found for {mission}: {e}")

    time.sleep(2)
    
    
    # Extract data from subpages
    for subpage in mission_subpages:
        subpage_url = f"{base_url}/{mission}-{subpage}"
        driver.get(subpage_url)

        try:
            # Wait for the content area to load
            content_area = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, ".field-item.even"))
            )
            print(f"Element found for: {mission}-{subpage}")

            # Extract data from the subpage
            subpage_data = {
                "text": content_area.text,
                "images": [],
                "tables": [],
                "pdfs": []
            }

            # Extract images
            images = content_area.find_elements(By.TAG_NAME, "img")
            for img in images:
                img_src = urljoin(base_url, img.get_attribute("src"))
                subpage_data["images"].append(img_src)

            # Extract tables
            tables = content_area.find_elements(By.TAG_NAME, "table")
            for table in tables:
                subpage_data["tables"].append(table.get_attribute("outerHTML"))

            # Extract PDF links
            pdf_links = content_area.find_elements(By.CSS_SELECTOR, "p a[href$='.pdf']")
            for pdf in pdf_links:
                relative_url = pdf.get_attribute("href")
                full_url = urljoin(base_url, relative_url)
                subpage_data["pdfs"].append(full_url)

            # Store extracted data
            mission_data_store[f"{mission}-{subpage}"] = subpage_data

        except Exception as e:
            print(f"Error: Element not found for {mission}-{subpage}: {e}")

        time.sleep(2)
        


# Close the browser
driver.quit()

Element found for: insat-3d
Element found for: insat-3d-introduction
Element found for: insat-3d-objectives
Element found for: insat-3d-spacecraft
Element found for: insat-3d-payloads
Element found for: insat-3d-references
Element found for: insat-3dr
Element found for: insat-3dr-introduction
Element found for: insat-3dr-objectives
Element found for: insat-3dr-spacecraft
Element found for: insat-3dr-payloads
Element found for: insat-3dr-references
Element found for: megha-tropiques
Element found for: megha-tropiques-introduction
Element found for: megha-tropiques-objectives
Element found for: megha-tropiques-spacecraft
Element found for: megha-tropiques-payloads
Element found for: megha-tropiques-references
Element found for: scatsat-1
Element found for: scatsat-1-introduction
Element found for: scatsat-1-objectives
Element found for: scatsat-1-spacecraft
Element found for: scatsat-1-payloads
Element found for: scatsat-1-references
Element found for: kalpana-1
Element found for: kalpan

In [15]:
with open("mosdac_mission_data.json", "w", encoding="utf-8") as json_file:
    json.dump(mission_data_store, json_file, ensure_ascii=False, indent=4)

print("\n✅ Data extraction completed! Saved as 'mosdac_mission_data.json'.")


✅ Data extraction completed! Saved as 'mosdac_mission_data.json'.
